In [45]:
import numpy as np
from scipy.spatial import ConvexHull
from matplotlib.path import Path
import time

# Fitness Function

In [46]:
def fitness_repel(pi, j, Dmin=2):
    """
    计算指定个体的COLLISION AVOIDANCE
    Input:
        pi: 子种群，Np*2维
        j: 指定个体索引
    """
    distances = np.linalg.norm(pi - pi[j], axis=1)
    distances[j] = np.inf # 排除自身
    nnd = np.min(distances)
    return np.exp(2 * (Dmin - nnd)) if nnd < Dmin else 1
    
def fitness_closure(firsts, idx, p_ij, prey):
    """
    计算CLOSURE
    Input:
        firsts: 每个子种群首元素，Ns*2维
        idx：指定子中群索引
        p_ij: 第i个子种群第j个个体位置，2维
        prey: 猎物位置，2维
    """
    if len(firsts) < 3:
        return 0
    firsts[idx] = p_ij # 替换为虚拟robot
    hull = ConvexHull(firsts)
    hull_vertices = firsts[hull.vertices]
    hull_path = Path(hull_vertices)
    if hull_path.contains_point(prey, radius=-1e-10): # 判断是否在凸包内部
        return 1
    def is_point_on_segment(point, p1, p2):
        v1 = point - p1
        v2 = point - p2
        if np.cross(np.append(v1, [0]), np.append(v2, [0]))[-1] == 0:  # 点在直线上
            if v1[0] * v2[0] <= 0 and v1[1] * v2[1] <= 0:  # 点在线段上
                return True
        return False
    for i in range(len(hull_vertices)):
        p1 = hull_vertices[i]
        p2 = hull_vertices[(i+1) % len(hull_vertices)]
        if is_point_on_segment(prey, p1, p2): # 判断是否凸包边界上
            return 0.5
    return 0

def fitness_expanse(firsts, idx, pij, prey):
    """
    计算SWARM EXPANSE
    Input:
        firsts: 每个子种群首元素，Ns*2维
        pij: 第i个子种群第j个个体位置，2维
        prey: 猎物位置，2维
    """
    firsts[idx] = pij # 替换为虚拟robot
    return np.sum(np.linalg.norm(firsts - prey, axis=1)) / len(firsts)

def fitness_uniformity(bins):
    """
    计算UNIFORMITY
    Input:
        bins = [[N11, N12], [N21, N22]]
    Output:
        std: 矩阵元素的标准差
    """ 
    return np.std(bins)


# Iteration

In [70]:
def ccpso(Ns=20, Np=10, left_range=-100, right_range=100, Tv=10, time_lim=1000):
    start_time = time.time()
    population = left_range + (right_range - left_range) * np.random.rand(Ns, Np, 2) # 初始种群
    
    def evaluate(idx):
        # TODO: 更新虚拟robot速度、位置，计算适应度
        ...
        
    population = np.array([[[1., 2.], [3., 4.]], [[1., 2.], [5., 6.]], [[2., 1.], [3., 4.]]])
    
    while time.time() - start_time < time_lim:
        for i in range(Ns):
            # 评估每个虚拟robot
            evaluate(i)
            
            # 虚拟robot重叠时重置
            unique_cnt = np.unique(population[i, 1:], axis=0).shape[0]
            if unique_cnt < Tv:
                population[i, 1:] = left_range + (right_range - left_range) * np.random.rand(Np - 1, 2)
                evaluate(i)
                
            # TODO: 更新真实robot速度、位置，计算适应度
            
            # TODO: 真实robot的fitness为subpopulation中最小时重置
            if ...:
                population[i, 1:] = left_range + (right_range - left_range) * np.random.rand(Np - 1, 2)
                evaluate(i)
            #  TODO: 死锁时添加随机扰动
            elif ...:
                population[i, 0] += np.random.normal(loc=0, scale=1, size=2)
                break
        
        # TODO: 局部最优时添加随机扰动
        if ...:
            population[:, 0] += np.random.normal(loc=0, scale=1, size=(Ns, 2))
           
        
ccpso(3, 2, time_lim=0.5)